# เอ็กเซปชัน

ข้อผิดพลาดในโปรแกรมเป็นสิ่งที่เรามักจะป้องกันได้โดยการตรวจสอบสถานะหรือค่าต่าง ๆ ก่อนที่จะดำเนินการต่อ เช่น ตรวจสอบค่าของพารามิเตอร์ที่รับมาว่ามีค่าอยู่ในช่วงที่ถูกต้องหรือไม่ ตรวจสอบค่าผลลัพธ์จากการเรียกเมทอดหรือการทำงานอื่น ๆ ว่าได้ผลลัพธ์ที่ถูกต้องหรือไม่ หรือตรวจสอบค่าของตัวแปรต่าง ๆ ก่อนที่จะดำเนินการว่าค่าเหล่านั้นอยู่ในสถานะที่จะดำเนินการต่อได้หรือไม่

เมื่อเราตรวจสอบแล้วพบว่ามีข้อผิดพลาด เรามีทางเลือกที่จะจัดการกับข้อผิดพลาดนั้นเดี๋ยวนั้น หรือจะรายงานต่อไปให้ส่วนอื่นของโปรแกรมเป็นผู้จัดการ

กรณีที่ข้อผิดพลาดนั้นจัดการได้ทันทีและสามารถดำเนินการต่อไปได้อย่างไม่มีปัญหาก็ควรจะจัดการเดี๋ยวนั้นเลย แต่ถ้าไม่สามารถจัดการได้ การรายงานต่อไปเป็นสิ่งที่ควรกระทำ

การรายงานข้อผิดพลาดต่อไปยังส่วนอื่นของโปรแกรมมีอยู่ 3 แนวทางหลัก ๆ

1. การคืนค่าเป็นค่าผลลัพธ์แสดงความผิดพลาด
2. การกำหนดค่าตัวแปรสถานะบางตัวเพื่อบ่งชี้ว่าเกิดความผิดพลาดขึ้น
3. การใช้กลไกของเอ็กเซปชัน (exception)

แบบที่ 1 และ 2 เป็นการรายงานข้อผิดพลาดแบบที่โค้ดส่วนที่รับข้อผิดพลาดสามารถละเลยได้โดยการไม่ตรวจสอบค่าที่คืนกลับมาหรือตัวแปรสถานะ ซึ่งอาจนำไปสู่ความผิดพลาดต่อเนื่องจากจุดนั้นได้ ทำให้การหาต้นตอของข้อผิดพลาดเมื่อปัญหาแสดงตัวขึ้นมาให้ผู้ใช้เห็นทำได้ยากขึ้น

แบบที่ 3 เป็นกลไกการรายงานข้อผิดพลาดแบบที่บังคับให้โค้ดที่รับข้อผิดพลาดต้องจัดการกับข้อผิดพลาด ไม่สามารถละเลยได้ การละเลยจะทำให้โปรแกรมหยุดทำงานพร้อมรายงานข้อผิดพลาดทันที ไม่ปล่อยให้โปรแกรมทำงานต่อไปในสถานะที่ผิดพลาด ซึ่งรูปแบบนี้เป็นแบบที่เราจะศึกษากันในบทนี้

ตัวอย่างบางส่วนของความผิดพลาดที่นำไปสู่การเกิดเอ็กเซปชันที่เรามักจะพบได้บ่อยได้แก่

- การพยายามใช้งานอ็อบเจกต์ด้วยตัวอ้างอิงที่มีค่าเป็น `null` (`NullPointerException`)
- การพยายามแปลงชนิดอ็อบเจกต์โดยที่ชนิดของอ็อบเจกต์นั้นไม่ได้มีความสัมพันธ์แบบ is-a กับชนิดปลายทางที่เราต้องการแปลง (`ClassCastException`)
- การอ้างอิงนอกขอบเขตสมาชิกของอาร์เรย์ (`ArrayIndexOutOfBoundsException`)
- การส่งอาร์กิวเมนต์ผิดรูปแบบที่เมทอดต้องการ (`IllegalArgumentException`)

การเกิดขึ้นของเอ็กเซปชันอาจจะเกิดจากระบบของ Java เองพบข้อผิดพลาดแล้วสร้างขึ้น หรือไลบรารีที่เราใช้อยู่พบลักษณะผิดปกติแล้วสร้างขึ้น หรือเราสร้างขึ้นเองในโค้ดของเราด้วยคำสั่ง `throw` (โยน)

เมื่อเกิดเอ็กเซปชันขึ้นแล้ว โค้ดของเราต้องจัดการกับเอ็กเซปชันนั้น ถ้าไม่จัดการ เอ็กเซปชันนั้นจะทำให้เมทอดที่กำลังทำงานอยู่จบการทำงานทันทีแล้วส่งเอ็กเซปชันนั้นไปยังเมทอดที่เรียกเมทอดนั้นอีกที กระบวนการนี้จะเกิดต่อกันเป็นทอดถ้าไม่มีโค้ดที่จัดการกับเอ็กเซปชันนั้นดักอยู่เลย จนไปถึงเมทอดสุดท้ายคือ `main()` หรือเมทอดเริ่มต้นในเทรดอื่น ถ้ายังไม่พบโค้ดที่จัดการกับเอ็กเซปชันนั้น โปรแกรมก็จะหยุดการทำงานแล้วแจ้งข้อผิดพลาด โดยแสดงรายการที่เรียกว่า stack trace ซึ่งจะระบุชื่อของเอ็กเซปชันและรายการเมทอดตั้งแต่จุดที่เกิดเอ็กเซปชันไปยังผู้เรียกเมทอดนั้นเป็นทอด ๆ จนมาถึงเมทอดตัวแรกสุด ซึ่งข้อมูลเหล่านี้จะมีประโยชน์ในการติดตามแก้ไขข้อผิดพลาด

โปรแกรมที่มีหลายเทรดจะหยุดทำงานเฉพาะเทรดที่เกิดเอ็กเซปชัน เทรดอื่น ๆ จะยังทำงานต่อไปได้

พิจารณาโค้ดต่อไปนี้

In [ ]:
import java.util.Scanner;

public class DivideByZeroNoExceptionHandling {
    public static int quotient(int numerator, int denominator) {
        return numerator / denominator;
    }
    
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        System.out.print("Enter an integer numerator: ");
        int numerator = input.nextInt();
        System.out.print("Enter an integer denominator: ");
        int denominator = input.nextInt();
        
        int result = quotient(numerator, denominator);
        System.out.printf("%d/%d = %d%n", numerator, denominator, result);
    }
}

เมทอด `quotient()` รับค่าจำนวนเต็ม 2 ค่าแล้วนำมาหารกัน ผลการรันปกติเป็นดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

ในกรณีที่เราป้อนข้อมูลค่าหนึ่งเป็นข้อความ จะเกิดเอ็กเซปชันขึ้น และเนื่องจากเราไม่ได้จัดการกับเอ็กเซปชัน โปรแกรมจึงหยุดทำงานและแสดง stack trace ออกมาดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: seven
Exception in thread "main" java.util.InputMismatchException
	at java.base/java.util.Scanner.throwFor(Scanner.java:860)
	at java.base/java.util.Scanner.next(Scanner.java:1497)
	at java.base/java.util.Scanner.nextInt(Scanner.java:2161)
	at java.base/java.util.Scanner.nextInt(Scanner.java:2115)
	at DivideByZeroNoExceptionHandling.main(DivideByZeroNoExceptionHandling.java:13)
```

เอ็กเซปชันนี้เกิดขึ้นเมื่อเราป้อนค่า `seven` ให้คำสั่ง `input.nextInt()` ซึ่งต้องการข้อมูลที่เป็นตัวเลขจำนวนเต็ม ข้อความจาก stack trace บอกเราว่าเกิดเอ็กเซปชันขึ้นโดยมีชนิดเป็น `InputMismatchException` เกิดขึ้นจากในคลาส `Scanner` ซึ่งเป็นไลบรารีของ Java เอง และเอ็กเซปชันได้ถูกส่งมาจนถึง `main()` ในบรรทัดที่ 13 ซึ่งเป็นการเรียกคำสั่ง `input.nextInt()` และเนื่องจากไม่มีการจัดการกับเอ็กเซปชัน โปรแกรมก็หยุดทำงานในที่สุด

ถ้าเราลองรันใหม่แล้วป้อนค่าตัวหารเป็น `0` ดู

```
Enter an integer numerator: 100
Enter an integer denominator: 0
Exception in thread "main" java.lang.ArithmeticException: / by zero
	at DivideByZeroNoExceptionHandling.quotient(DivideByZeroNoExceptionHandling.java:5)
	at DivideByZeroNoExceptionHandling.main(DivideByZeroNoExceptionHandling.java:15)
```

จะพบว่าเกิด `ArithmeticException` ขึ้น ซึ่งในกรณีนี้เอ็กเซปชันถูกสร้างขึ้นและส่งมาโดยระบบ runtime ของ Java เองจากการที่เราหารจำนวนเต็มด้วย `0`

ในกรณีนี้ stack trace แจ้งว่าเอ็กเซปชันเกิดขึ้นที่บรรทัดที่ 5 ในโปรแกรมของเราในเมทอด `quotient()` และส่งต่อมาที่ `main()` และจบการทำงานลง

## การจัดการกับเอ็กเซปชันเบื้องต้น

เราสามารถจัดการกับเอ็กเซปชันในตัวอย่างนี้ได้โดยใช้คำสั่ง `try` ... `catch` ดังตัวอย่างที่ปรับปรุงใหม่นี้

In [ ]:
import java.util.Scanner;
import java.util.InputMismatchException;

public class DivideByZeroWithExceptionHandling {
    public static int quotient(int numerator, int denominator) {
        return numerator / denominator;
    }
    
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        boolean keepRetrying = true;
        
        do {
            try {
                System.out.print("Enter an integer numerator: ");
                int numerator = input.nextInt();
                System.out.print("Enter an integer denominator: ");
                int denominator = input.nextInt();

                int result = quotient(numerator, denominator);
                System.out.printf("%d/%d = %d%n", numerator, denominator, result);

                keepRetrying = false;
            } catch (InputMismatchException e) {
                System.err.printf("Exception: %s%n", e);
                System.out.println("You must enter an integer. Please try again.");
                input.nextLine(); // discard the input and try again
            } catch (ArithmeticException e) {
                System.err.printf("Exception: %s%n", e);
                System.out.println("Denominator cannot be zero. Please try again.");
            }
        } while (keepRetrying);

    }
}

เราเพิ่มลูป `do` ... `while` เพื่อให้เกิดการป้อนซ้ำจนกว่าป้อนค่าถูกต้องทั้ง 2 ค่า โดยใช้ตัวแปร `keeyRetrying` เป็นตัวกำหนด ถ้าโปรแกรมสามารถทำงานมาได้จนถึงบรรทัดที่กำหนด `keepRetrying = false` ก็จะออกจากลูปโดยไม่ต้องทำซ้ำรอบต่อไป

โค้ดเดิมเราใส่ไว้ในบล็อก `try` และมีโค้ดจัดการเอ็กเซปชันเพิ่มขึ้นมาในบล็อก `catch` โดยทั้งหมดนี้จะอยู่ในลูป `do` ... `while` เพื่อให้เกิดการทำใหม่เมื่อมีข้อผิดพลาด

ผลการรันปกติเป็นดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 7    
100/7 = 14
```

ในกรณีที่เราป้อนข้อมูลเป็นข้อความ จะเกิด `InputMismatchException` ขึ้นที่คำสั่ง `input.nextLine()` ในบล็อก `try` การเกิดเอ็กเซปชันในบล็อก `try` จะทำให้โปรแกรมมีการเทียบเอ็กเซปชันนั้นกับบล็อก `catch` ทันที ถ้ามีการประกาศบล็อก `catch` ที่มีชนิดเอ็กเซปชันสอดคล้องกัน โปรแกรมก็จะไปทำงานที่บล็อก `catch` นั้น โดยเอ็กเซปชันที่เกิดขึ้นนั้นจะถูกจับไว้ด้วยตัวแปรที่ประกาศที่บล็อก `catch` ซึ่งในที่นี้ก็คือ `e`

```
Enter an integer numerator: 100
Enter an integer denominator: seven
Exception: java.util.InputMismatchException
You must enter an integer. Please try again.
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

ในบล็อก `catch` นี้ เราสั่งให้แสดงข้อมูลของเอ็กเซปชันและข้อความแสดงความผิดพลาด และเรียก `input.nextLine()` เพื่อล้างข้อความที่ค้างอยู่ที่อินพุตเนื่องจากเกิดการอ่านที่ผิดพลาด

ลองรันใหม่อีกครั้งแล้วป้อนค่าตัวหารเป็น 0 ได้ผลดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 0
Exception: java.lang.ArithmeticException: / by zero
Denominator cannot be zero. Please try again.
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

เกิด `ArithmeticException` ที่เมทอด `quotient()` ซึ่งถูกเรียกจากในบล็อก `try` ของ `main()` ในกรณีนี้ ตัวเมทอด `quotient()` เองไม่ได้จัดการกับเอ็กเซปชันนั้น เอ็กเซปชันจึงถูกส่งต่อมาที่ผู้ที่เรียก `quotient()` ซึ่งก็คือในส่วนของ `main()` และถูกดักไว้ด้วยบล็อก `catch` ที่ตรงกัน ซึ่งจะแสดงข้อผิดพลาดแล้วกลับไปรับค่าใหม่

สังเกตว่าเมื่อเกิดเอ็กเซปชันขึ้นในบล็อก `try` โค้ดในบล็อกนี้จะไม่ทำงานต่อ ทำให้ไปไม่ถึงคำสั่ง `keepRetrying = false` ค่าของ `keepRetrying` จึงยังเป็น `true` และทำให้ลูปกลับไปวนซ้ำรับค่าใหม่

จากตัวอย่างนี้เราสรุปการจัดการกับเอ็กเซปชันเบื้องต้นได้ดังนี้

1. ใส่โค้ดที่ต้องการให้ทำงานและมีความเสี่ยงที่จะเกิดเอ็กเซปชันขึ้นในบล็อก `try` เมื่อเกิดเอ็กเซปชันขึ้นที่จุดใดในบล็อก `try` โปรแกรมจะไม่ทำงานต่อในโค้ดส่วนที่เหลือในบล็อก
2. กำหนดบล็อก `catch` ที่จะจัดการกับเอ็กเซปชันที่จะเกิดขึ้น โดยการระบุชนิดของเอ็กเซปชันและชื่อตัวแปรที่จะใช้แทนเอ็กเซปชันนั้นถ้าเกิดเอ็กเซปชันขึ้น เราเรียกส่วนประกาศในวงเล็บของ `catch` นั้นว่าพารามิเตอร์ของเอ็กเซปชัน เอ็กเซปชันถือเป็นอ็อบเจกต์ชนิดหนึ่ง เราสามารถจะรับมาเพื่อนำมาจัดการต่อได้
3. ถ้าบล็อก `try` ทำงานสำเร็จตามปกติ หรือถ้าเกิดเอ็กเซปชันขึ้นและทำในบล็อก `catch` เสร็จสิ้นแล้ว โปรแกรมจะทำงานต่อในส่วนที่ต่อจากบล็อก `catch` สุดท้าย

เราสามารถกำหนดบล็อก `catch` กี่บล็อกก็ได้ตามจำนวนชนิดของเอ็กเซปชันที่เราต้องการจัดการ เอ็กเซปชันที่เกิดขึ้นอาจมีชนิดไม่ตรงกับที่ประกาศในบล็อก `catch` แต่ถ้าเอ็กเซปชันที่เกิดขึ้นเป็นซับคลาสของเอ็กเซปชันชนิดที่ประกาศ บล็อก `catch` นั้นก็จะจับเอ็กเซปชันนั้นได้เช่นกัน

เราสามารถประกาศบล็อก `catch` บล็อกเดียวที่จัดการกับเอ็กเซปชันหลายชนิดได้ด้วยรูปแบบนี้

```
catch (Exception1 | Exception2 | Exception3 e) {
    // ...
}
```

โดย _Exception1_, _Exception2_ และ _Exception3_ เป็นชนิดของเอ็กเซปชันที่ต้องการจับ

ในกรณีที่ เอ็กเซปชันที่เกิดขึ้นไม่เข้ากับบล็อก `catch` ใดเลย เอ็กเซปชันนั้นจะถูกส่งต่อไปยังเมทอดที่เรียกเมทอดนี้อีกทอดหนึ่งตามขั้นตอน

> **หมายเหตุ** เอ็กเซปชันทุกชนิดเป็นซับคลาสของ `Throwable` เราสามารถใช้ `catch` กับคลาสชนิดใดก็ได้ที่เป็นซับคลาสของ `Throwable` เราสามารถสร้างคลาสเอ็กเซปชันของเราเองได้เช่นกัน

## ลำดับชั้นของคลาสเอ็กเซปชัน

เอ็กเซปชันทุกชนิดเป็นซับคลาสของคลาส `Exception` ซึ่งเป็นซับคลาสของ `Throwable` อีกที อ็อบเจกต์ที่ใช้ในกลไกจัดการเอ็กเซปชันได้ต้องเป็นอ็อบเจกต์จากคลาสหรือซับคลาสของ `Throwable` เท่านั้น

ซับคลาสโดยตรงของ `Throwable` จะประกอบด้วย `Exception` และ `Error` และมีโครงสร้างของลำดับชั้นดังนี้

![ExceptionHierarchy](ExceptionHierarchy.png)

Java แบ่งลักษณะอาการผิดปกติไว้เป็นแบบร้ายแรงซึ่งอยู่ในกลุ่มคลาส `Error` และแบบที่จัดการได้ซึ่งอยู่ในกลุ่มคลาส `Exception`

กลุ่มคลาส `Error` จะเป็นลักษณะความผิดปกติที่เกิดขึ้นใน JVM และทำให้ไม่สามารถทำงานต่อและกลับเข้าสู่สภาวะปกติได้ เช่น `StackOverflowError` หรือ `OutOfMemoryError` ซึ่งความผิดพลาดลักษณะนี้โดยปกติแล้วจะร้ายแรงเกินกว่าที่โปรแกรมจะจัดการได้

กลุ่มคลาส `Exception` เป็นลักษณะความผิดปกติที่จัดการได้ โดยยังแบ่งได้เป็นเอ็กเซปชันแบบที่ถูกตรวจสอบ (checked exception) กับแบบที่ไม่ถูกตรวจสอบ (unchecked exception) จากแผนผังลำดับชั้นของคลาสเอ็กเซปชันนี้ คลาสที่เป็นอยู่ในกลุ่มของ `RuntimeException` (เป็นซับคลาสของ `RuntimeException`) จะเป็น unchecked exception ส่วนคลาสเอ็กเซปชันที่ไม่ได้อยู่ในกลุ่ม `RuntimeException` จะจัดเป็น checked exception

กลุ่มที่เป็น checked exeption มักจะเป็นข้อผิดพลาดในลักษณะที่อยู่นอกเหนือการควบคุมของโปรแกรม เช่น เปิดไฟล์ไม่ได้ หรือไฟล์ไม่มีอยู่ ซึ่งจะเกิดเป็น `IOException` หรือ `FileNotFoundException` เป็นต้น เอ็กเซปชันกลุ่มนี้ คอมไพเลอร์ Java จะบังคับให้เราเลือกระหว่างการจัดการกับเอ็กเซปชันหรือการประกาศส่งต่อเอ็กเซปชัน ถ้าเราไม่ต้องการจัดการกับเอ็กเซปชันนี้ในเมทอดของเรา เราต้องประกาศว่าเมทอดของเราอาจจะส่งต่อเอ็กเซปชันนี้ได้ ดังตัวอย่างโค้ดนี้

In [ ]:
public Image loadImage(String fileName) throws IOException {
    return ImageIO.read(new File(fileName));
}

การเรียก `ImageIO.read()` อาจทำให้เกิด `IOException` ซึ่งเป็น checked exception ได้ เมทอด `loadImage()` มีทางเลือก 2 ทาง ทางแรกคือจัดการกับเอ็กเซปชันนี้เองโดยใช้ `try` ... `catch` หรือทางที่สอง คือการประกาศว่าเมทอดอาจจะส่งเอ็กเซปชันนี้ออกมาได้ โดยการประกาศ `throws` เอาไว้ที่ส่วนประกาศเมทอด

คอมไพเลอร์ Java จะบังคับให้เราเลือกอย่างใดอย่างหนึ่งในกรณีของ checked exception ซึ่งถ้าเราไม่ทำตามนี้ โปรแกรมก็จะคอมไพล์ไม่ผ่าน

เอ็กเซปชันประเภท checked exception นี้ถ้าเราไม่จัดการเลย จะทำให้เกิดการประกาศ `throws` ต่อกันเป็นทอด เมทอดที่เรียกใช้ `loadImage()` นี้ถ้าไม่จัดการกับเอ็กเซปชันก็ต้องประกาศ `throws` ด้วย และเมทอดที่เรียกเมทอดนี้ต่อก็เช่นกัน

อีกประเภทคือ unchecked exception ซึ่งเป็นกลุ่มที่เป็นซับคลาสของ `RuntimeException` มักจะเป็นความผิดพลาดที่ป้องกันได้ เช่น `ArithmeticException` หรือ `NullPointerException` ซึ่งป้องกันได้โดยการตรวจสอบค่าก่อนการใช้งาน เอ็กเซปชันประเภทนี้คอมไพเลอร์จะไม่บังคับให้เราจัดการหรือประกาศในส่วนประกาศของเมทอด ดังจะเห็นได้จากตัวอย่างเมทอด `quotient()` ในตอนต้นของบทนี้ซึ่งสามารถจะทำให้เกิด `ArithmeticException` จากการหารด้วย `0` ได้ แต่ตัวเมทอดก็ไม่ได้จัดการหรือประกาศส่งต่อแต่อย่างใด

## การโยนเอ็กเซปชัน

นอกเหนือจากเอ็กเซปชันที่เกิดขึ้นจาก JVM และจากการเรียกใช้งานไลบรารีต่าง ๆ แล้ว เรายังสามารถสร้างเอ็กเซปชันขึ้นมาเองได้เช่นกัน ซึ่งเรามักจะใช้กับความผิดพลาดที่เรากำหนดขึ้นเองในโปรแกรมของเรา

เราสามารถสร้างอ็อบเจกต์เอ็กเซปชันแล้วส่งต่อ (เรียกว่าโยนหรือ throw) ได้ด้วยคำสั่ง `throw` ดังนี้

In [ ]:
public Image loadImage(String fileName) throws IOException {
    Image image = ImageIO.read(new File(fileName));
    if (image == null)
        throw new IOException("Unrecognized image type");
    return image;
}

ในตัวอย่างนี้ ถ้าเราพบว่าค่าที่ส่งกลับมาจาก `ImageIO.read()` เป็น `null` เราจะสร้างอ็อบเจกต์เอ็กเซปชันชนิด `IOException` ขึ้นด้วยคำสั่ง `new` พร้อมกับข้อความว่า `"Unrecognized image type"` แล้ว `throw` เอ็กเซปชันนี้ออกไปนอกเมทอดไปยังผู้เรียกใช้เมทอดนี้

## บล็อก finally

In [4]:
public class UsingExceptions {
    public static void main(String[] args) {
        try {
            throwException();
        } catch (Exception e) {
            System.err.println("Exception handled in main()");
        }
    }
}

## การสร้างคลาสเอ็กเซปชัน

### การโยนเอ็กเซปชันต่อ

## รายการย้อนรอยสแต็ก

## การใช้ assert